In [ ]:
import os
import numpy as np
import cv2 as cv
import tqdm
from matplotlib import pyplot as plt
import json

MIN_MATCH_COUNT = 10

TEMPLATES_PATH = 'templates/'
IMAGES_PATH = 'dataset/'
DATASET_JSON = 'dataset.json'

# Initiate SIFT detector
sift = cv.SIFT_create()

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv.FlannBasedMatcher(index_params, search_params)

def get_RGB_from_BGR(img):
	return cv.cvtColor(img, cv.COLOR_BGR2RGB)

templates = {}
# Find all images in the templates folder
for path in os.listdir(TEMPLATES_PATH):
	if not path == '.DS_Store':
		templates[path] = {}

images = {}
# Read dataset.json file
with open(DATASET_JSON) as json_file:
	for tp in json.load(json_file)['true_positives']:
		images[tp['image']] = {
			'templates' : tp['templates']
		}

print(f"Found {len(templates)} templates and {len(images)} images")

In [ ]:
def center_crop(img, factor=0.97):
	# Cropping because many templates have angle brackets at the corners
	
	height, width = img.shape[:2]
	# Calculate new dimensions
	new_height = int(height * 0.97)
	new_width = int(width * 0.97)

	# Calculate top-left corner and bottom-right corner for cropping
	start_x = (width - new_width) // 2
	start_y = (height - new_height) // 2
	end_x = start_x + new_width
	end_y = start_y + new_height

	# Crop the image
	return img[start_y:end_y, start_x:end_x]

In [ ]:
for template in tqdm.tqdm(templates):
	template_img = center_crop(cv.imread(TEMPLATES_PATH + template))
	kp_template, des_template = sift.detectAndCompute(template_img, None)
	templates[template]['keypoints'] = kp_template
	templates[template]['descriptors'] = des_template

In [ ]:
for image in tqdm.tqdm(images):
	kp_image, des_image = sift.detectAndCompute(cv.imread(IMAGES_PATH + image), None)
	images[image]['keypoints'] = kp_image
	images[image]['descriptors'] = des_image

In [ ]:
def get_draw(kp_t, kp_img, template, full_img, good):
	src_pts = np.float32([ kp_t[m.queryIdx].pt for m in good]).reshape(-1,1,2)
	dst_pts = np.float32([ kp_img[m.trainIdx].pt for m in good]).reshape(-1,1,2)
	M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
	matchesMask = mask.ravel().tolist()
	template_img = cv.imread(template)
	img_img = cv.imread(full_img)
	h,w = template_img.shape[:2]
	pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
	dst = cv.perspectiveTransform(pts,M)
	img2 = cv.polylines(img_img,[np.int32(dst)],True,255,3, cv.LINE_AA)
	draw_params = dict(matchColor = (0,255,0), # draw matches in green color
				singlePointColor = None,
				matchesMask = matchesMask, # draw only inliers
				flags = 2)
	match_img = cv.drawMatches(template_img,kp_t,img2,kp_img,good,None,**draw_params)
	return match_img

def draw_match(match_img):
	plt.imshow(get_RGB_from_BGR(match_img))

In [ ]:
def search_match(image, template, dist_max=0.7, save_draw=False, draw_on_found=False):
	kp_t, des_t = templates[template]['keypoints'], templates[template]['descriptors']
	kp_img, des_img = images[image]['keypoints'], images[image]['descriptors']

	should_be_found = template in images[image]['templates']

	matches = flann.knnMatch(des_t,des_img,k=2)
	good = []
	for m,n in matches:
		if m.distance < dist_max*n.distance:
			good.append(m)

	if len(good)>MIN_MATCH_COUNT:
		outcome = 'TP' if should_be_found else 'FP'
		print(f"{outcome} found for {template} on image {image} with {len(good)} matches out of {MIN_MATCH_COUNT}")

		if save_draw:
			try:
				draw = get_draw(kp_t, kp_img, TEMPLATES_PATH+template, IMAGES_PATH+image, good)
			except:
				print(f"[ERROR] Couldn't draw match for {template} on image {image} Skipping this draw")
				draw = None
			
			if draw_on_found:
				draw_match(draw)
			return draw, outcome
	else:
		outcome = 'TN' if not should_be_found else 'FN'
		print(f"{outcome} found for {template} on image {image} with {len(good)} matches out of {MIN_MATCH_COUNT}")
		return None, outcome

In [ ]:
results = {
	'TP' : [],
	'FP' : [],
	'TN' : [],
	'FN' : []
}

for image in images:
	for template in templates:
		draw, outcome = search_match(image, template, save_draw=True)
		results[outcome].append("." if draw is None else draw)

print(f"TP: {len(results['TP'])}, FP: {len(results['FP'])}, TN: {len(results['TN'])}, FN: {len(results['FN'])}")

precision = len(results['TP']) / (len(results['TP']) + len(results['FP']))
recall = len(results['TP']) / (len(results['TP']) + len(results['FN']))
f1 = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")